In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import time
import pandas as pd
from intermediates import *

In [2]:
import lsst.sims.maf.slicers as slicers
from lsst.obs.lsstSim import LsstSimMapper
from lsst.sims.utils import ObservationMetaData
from lsst.sims.coordUtils import chipNameFromRaDec

In [3]:
#help(ObservationMetaData)
#help(chipNameFromRaDec)

In [4]:
dbpath= '/Volumes/Transcend/minion_1016_sqlite.db'
#simdata= getSimData(dbpath, 'r')   # get the OpSim columns

In [5]:
def findChips(dpbath, focusDither, focusID, 
              filterBand= 'r', disc= True):
    
    FOV_radius= 0.0305
    nside= 512
    printProgress('Getting simData ... ', highlight= True)
    simdata= getSimData(dbpath, filterBand, ['expDate', 'obsHistID'])
    
    printProgress('Finding region pixels ... ', highlight= True)
    centralRA, centralDec, regionPixels= findRegionPixels(focusID, simdata,
                                                          nside, disc, FOV_radius)
    totPixels= len(regionPixels)
    printProgress('Total number of pixels: %d'%(totPixels))
                  
    # set up the slicer
    hpSlicer= slicers.HealpixSlicer(nside= nside)
    hpSlicer.setupSlicer(simdata)    # need to slice data?
    
    camera = LsstSimMapper().camera
    chipNames= []
    obsIDs= []
    expDates= []
    fIDs= []
    
    prevPercent= 0.
    for p, pixel in enumerate(regionPixels):
        pixRA, pixDec= hpSlicer._pix2radec(pixel)    # radians returned    
        indObsInPixel = hpSlicer._sliceSimData(pixel)
        
        for index in indObsInPixel['idxs']:
            # get data
            # for identifying each visit
            expDate= simdata[index]['expDate']
            obsID= simdata[index]['obsHistID']
            fID= simdata[index]['fieldID']
            
            # for chip finding
            if (focusDither=='NoDither'):
                pointingRA= np.degrees(simdata[index]['fieldRA'])   # need in degres
                pointingDec= np.degrees(simdata[index]['fieldDec']) # need in degres
            else:
                pointingRA= np.degrees(simdata[index]['ditheredRA']) # need in degres
                pointingDec= np.degrees(simdata[index]['ditheredDec']) # need in degres
            rotSkyPos= np.degrees(simdata[index]['rotSkyPos']) # need in degres
            expMJD= simdata[index]['expMJD']
            
            # set up for the finding the chips
            obs = ObservationMetaData(pointingRA= pointingRA, pointingDec= pointingDec,
                                      rotSkyPos= rotSkyPos, mjd= expMJD)
            chipsInVisit= chipNameFromRaDec(np.degrees(pixRA), np.degrees(pixDec),
                                            camera=camera, obs_metadata=obs)
            if chipsInVisit is not None:
                obsIDs.append(obsID)
                expDates.append(expDate)
                chipNames.append(chipsInVisit)
                fIDs.append(fID)
        percentDone= 100.*(p+1)/totPixels
        
        delPercent= percentDone-prevPercent
        if (delPercent>5):
            printProgress('Percent pixels done: %f'%(percentDone))
            prevPercent= percentDone
        #if (percentDone>1.):
        #    return [np.array(obsIDs), np.array(expDates), np.array(fIDs), np.array(chipNames)]
    return [np.array(obsIDs), np.array(expDates), np.array(fIDs), np.array(chipNames)]

In [6]:
startTime= time.time()
obsIDs, expDates, fIDs, chipNames= findChips(dbpath, '', 1447,
                                              filterBand= 'r', disc= True)
printProgress('Time Taken: %f'%(time.time()-startTime))


#############################################
## Getting simData ... 

#############################################
## Finding region pixels ... 
#
## Total number of pixels: 4568
Healpix slicer using NSIDE=512, approximate resolution 6.870973 arcminutes


/Users/Humna/lsst/DarwinX86/sims_utils/12.0-13-g6534730/python/lsst/sims/utils/ModifiedJulianDate.py:188: UTCtoUT1Warning: UTC is outside of IERS table for UT1-UTC.
Returning UT1 = UTC for lack of a better idea
This warning was caused by calling ModifiedJulianDate.dut1

  category=UTCtoUT1Warning)


#
## Percent pixels done: 5.013135


/Users/Humna/lsst/DarwinX86/sims_coordUtils/12.0-4-g7de6cfa+29/python/lsst/sims/coordUtils/CameraUtils.py:325: MultipleChipWarning: An object has landed on multiple chips.  You asked for this not to happen.
We will return only one of the chip names.  If you want both, try re-running with the kwarg allow_multiple_chips=True.
Offending chip names were ['R:4,0 S:0,2,A', 'R:4,0 S:0,2,B']
Offending pupil coordinate point was 0.019390737213 -0.020521799440

  category=MultipleChipWarning)


#
## Percent pixels done: 10.026270
#
## Percent pixels done: 15.039405
#
## Percent pixels done: 20.052539
#
## Percent pixels done: 25.065674
#
## Percent pixels done: 30.078809
#
## Percent pixels done: 35.091944
#
## Percent pixels done: 40.105079
#
## Percent pixels done: 45.118214
#
## Percent pixels done: 50.131349
#
## Percent pixels done: 55.144483
#
## Percent pixels done: 60.157618
#
## Percent pixels done: 65.170753
#
## Percent pixels done: 70.183888


/Users/Humna/lsst/DarwinX86/sims_coordUtils/12.0-4-g7de6cfa+29/python/lsst/sims/coordUtils/CameraUtils.py:325: MultipleChipWarning: An object has landed on multiple chips.  You asked for this not to happen.
We will return only one of the chip names.  If you want both, try re-running with the kwarg allow_multiple_chips=True.
Offending chip names were ['R:4,0 S:0,2,A', 'R:4,0 S:0,2,B']
Offending pupil coordinate point was 0.021111644894 -0.020519322438

  category=MultipleChipWarning)


#
## Percent pixels done: 75.197023
#
## Percent pixels done: 80.210158
#
## Percent pixels done: 85.223292
#
## Percent pixels done: 90.236427
#
## Percent pixels done: 95.249562
#
## Time Taken: 13487.972744


In [7]:
pd.DataFrame([obsIDs, expDates, fIDs, chipNames], ['obsHistID', 'expDate', 'fIDs', 'Chips'])

,0,1,2,3,4,5,6,7,8,9,...,660345,660346,660347,660348,660349,660350,660351,660352,660353,660354
obsHistID,2168708,1499774,1993954,456925,1993992,2182701,1299264,1700145,1499816,1422764,...,1195502,985809,482316,996583,237420,2396114,1487703,954630,777491,703076
expDate,276072453,192253838,254460584,57044790,254462202,277800203,165286880,219217178,192255814,181896202,...,151308376,125462727,60752445,126855059,28266491,308216066,190706460,120723136,98854581,89538853
fIDs,1564,1568,1568,1568,1568,1568,1568,1568,1568,1568,...,1220,1220,1220,1220,1220,1220,1220,1220,1220,1220
Chips,"R:2,1 S:0,2","R:2,1 S:2,2","R:0,2 S:0,0","R:1,0 S:1,2","R:0,1 S:1,0","R:1,4 S:0,0","R:0,1 S:1,2","R:1,3 S:1,1","R:2,1 S:2,2","R:2,4 S:2,1",...,"R:0,3 S:2,2","R:3,3 S:1,1","R:3,4 S:0,2","R:0,1 S:2,2","R:3,4 S:1,0","R:1,4 S:1,1","R:2,0 S:1,1","R:1,4 S:0,1","R:2,1 S:1,2","R:4,1 S:1,2"


In [8]:
print len(chipNames)
print len(np.unique(chipNames))
print len(np.unique(expDates))
print len(np.unique(obsIDs))

660355
205
2779
2779


In [9]:
obsIDsList, expDatesList, fIDsList, chipNamesList= [], [], [], []

for obs in np.unique(obsIDs):
    obsIDsList.append(obs)
    ind= np.where(obsIDs==obs)[0]
    
    expDatesList.append(np.unique(expDates[ind]))
    
    fIDsList.append(np.unique(fIDs[ind]))
    chipNamesList.append(np.unique(chipNames[ind]))

In [10]:
len(obsIDsList)

2779

In [11]:
pd.DataFrame([obsIDsList, expDatesList, fIDsList, chipNamesList], ['obsHistID', 'expDate', 'fIDs', 'Chips'])

,0,1,2,3,4,5,6,7,8,9,...,2769,2770,2771,2772,2773,2774,2775,2776,2777,2778
obsHistID,40336,40337,40338,40345,40366,40367,40368,40375,193115,193156,...,2440035,2440067,2440068,2440082,2440083,2440095,2441917,2441931,2441963,2441975
expDate,[4673449],[4673487],[4673526],[4673806],[4674632],[4674670],[4674709],[4674986],[22405149],[22406794],...,[314253049],[314254567],[314254614],[314255232],[314255281],[314256084],[314506587],[314507151],[314508445],[314508970]
fIDs,[1305],[1323],[1333],[1212],[1305],[1323],[1333],[1212],[1305],[1305],...,[1212],[1564],[1568],[1234],[1220],[1212],[1542],[1305],[1542],[1305]
Chips,"[R:1,1 S:2,1, R:1,1 S:2,2, R:1,2 S:2,0, R:2,1 ...","[R:1,0 S:2,1, R:1,0 S:2,2, R:1,1 S:2,0, R:1,1 ...","[R:1,0 S:2,0, R:1,0 S:2,1, R:1,0 S:2,2, R:1,1 ...","[R:3,3 S:1,0, R:3,3 S:1,1, R:3,3 S:1,2, R:3,3 ...","[R:1,1 S:2,2, R:1,2 S:2,0, R:2,1 S:0,1, R:2,1 ...","[R:1,0 S:2,1, R:1,0 S:2,2, R:1,1 S:2,0, R:1,1 ...","[R:1,0 S:2,1, R:1,0 S:2,2, R:1,1 S:2,0, R:1,1 ...","[R:3,3 S:1,0, R:3,3 S:1,1, R:3,3 S:1,2, R:3,3 ...","[R:0,0 S:2,1, R:1,0 S:0,0, R:1,0 S:0,1, R:1,0 ...","[R:0,0 S:2,1, R:1,0 S:0,0, R:1,0 S:0,1, R:1,0 ...",...,"[R:0,1 S:0,1, R:0,1 S:0,2, R:0,1 S:1,1, R:0,1 ...","[R:2,0 S:2,0, R:2,0 S:2,1, R:3,0 S:0,0, R:3,0 ...","[R:3,0 S:0,0, R:3,0 S:0,1, R:3,0 S:1,0, R:3,0 ...","[R:1,3 S:2,0, R:1,3 S:2,1, R:1,3 S:2,2, R:2,1 ...","[R:0,2 S:0,2, R:0,2 S:1,0, R:0,2 S:1,1, R:0,2 ...","[R:2,1 S:1,2, R:2,1 S:2,1, R:2,1 S:2,2, R:2,2 ...","[R:4,3 S:2,2]","[R:1,1 S:0,2, R:1,1 S:1,1, R:1,1 S:1,2, R:1,1 ...","[R:4,3 S:2,2, R:4,4 S:1,0]","[R:0,2 S:2,0, R:1,1 S:0,2, R:1,1 S:1,1, R:1,1 ..."


In [12]:
import pickle

dataToSave = {'obsHistID': obsIDsList, 'expDate': expDatesList, 'fIDs': fIDsList, 'chipNames': chipNamesList}

with open('chipPerVisitData.pickle', 'wb') as handle:
    pickle.dump(dataToSave, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
#with open('chipPerVisitData.pickle', 'rb') as handle:
#      savedData= pickle.load(handle)


In [15]:
minInd, maxInd= 0, 3
pd.DataFrame([obsIDsList[minInd:maxInd], expDatesList[minInd:maxInd], 
              fIDsList[minInd:maxInd], chipNamesList[minInd:maxInd]], ['obsHistID', 'expDate', 'fIDs', 'Chips'])

,0,1,2
obsHistID,40336,40337,40338
expDate,[4673449],[4673487],[4673526]
fIDs,[1305],[1323],[1333]
Chips,"[R:1,1 S:2,1, R:1,1 S:2,2, R:1,2 S:2,0, R:2,1 ...","[R:1,0 S:2,1, R:1,0 S:2,2, R:1,1 S:2,0, R:1,1 ...","[R:1,0 S:2,0, R:1,0 S:2,1, R:1,0 S:2,2, R:1,1 ..."


In [17]:
len(chipNamesList[0])

72

In [19]:
len(np.unique(chipNamesList[0]))

72